In [159]:
import pandas as pd
import os

# Voting data

In [160]:
folder = "data"
subfolder_vot = "voting"

path_vot = os.path.join(folder, subfolder_vot)
file_path_gen = os.path.join(path_vot, "pst4.csv")
file_path_gran = os.path.join(path_vot, "pst4p.csv")

data_gen = pd.read_csv(file_path_gen)
data_gran = pd.read_csv(file_path_gran)

In [161]:
print("General data")
print("length", len(data_gen))
print("unique okrsek", len(data_gen['OKRSEK'].unique()))
print("unique obec", len(data_gen['OBEC'].unique()))
print("unique okres", len(data_gen['OKRES'].unique()))
print()
print("Granular data")
print("length", len(data_gran))
print("unique okrsek", len(data_gran['OKRSEK'].unique()))
print("unique obec", len(data_gran['OBEC'].unique()))
print("unique okres", len(data_gran['OKRES'].unique()))
print()
print("Sanity checks")
print(len(data_gen['OKRSEK'].unique()) == len(data_gran['OKRSEK'].unique()))
print(len(data_gen['OBEC'].unique()) == len(data_gran['OBEC'].unique()))
print(len(data_gen['OKRES'].unique()) == len(data_gran['OKRES'].unique()))

General data
length 14819
unique okrsek 1502
unique obec 6389
unique okres 78

Granular data
length 179877
unique okrsek 1502
unique obec 6389
unique okres 78

Sanity checks
True
True
True


In [162]:
df_gen = data_gen[['ID_OKRSKY', 'OKRSEK', 'OBEC', 'OKRES', 'ODEVZ_OBAL', 'PL_HL_CELK']][data_gen['TYP_FORM'] == 1]
df_gen.head()

,ID_OKRSKY,OKRSEK,OBEC,OKRES,ODEVZ_OBAL,PL_HL_CELK
0,1,1,500011,7204,508,502
1,2,2,500011,7204,303,301
2,3,3,500011,7204,298,296
3,4,1,500020,7105,167,163
4,5,2,500020,7105,517,515


In [163]:
df_gran = data_gran[['ID_OKRSKY', 'OKRSEK', 'OBEC', 'OKRES', 'KSTRANA', 'POC_HLASU']][data_gran['TYP_FORM'] == 2]
df_gran.head()

,ID_OKRSKY,OKRSEK,OBEC,OKRES,KSTRANA,POC_HLASU
0,1,1,500011,7204,3,2
1,1,1,500011,7204,4,1
2,1,1,500011,7204,6,41
3,1,1,500011,7204,8,5
4,1,1,500011,7204,10,4


In [164]:
df_okrsek_votes = df_gran.pivot_table(
    index="ID_OKRSKY", columns="KSTRANA", values="POC_HLASU", aggfunc="sum", fill_value = 0
)
df_okrsek = pd.merge(df_okrsek_votes, df_gen, how='inner', on= 'ID_OKRSKY')
df_okrsek = df_okrsek[~(df_okrsek['OKRES'] == 9999)]

len(df_okrsek) == len(df_okrsek_votes)
df_okrsek.head()

,ID_OKRSKY,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,OKRSEK,OBEC,OKRES,ODEVZ_OBAL,PL_HL_CELK
0,1,0,0,2,1,0,41,0,5,0,...,159,58,0,23,2,1,500011,7204,508,502
1,2,0,0,0,0,0,34,2,8,0,...,78,42,0,8,0,2,500011,7204,303,301
2,3,0,1,0,1,0,18,1,5,0,...,88,38,0,13,1,3,500011,7204,298,296
3,4,0,0,0,0,0,20,0,2,1,...,69,18,0,6,0,1,500020,7105,167,163
4,5,0,2,0,0,1,67,0,7,0,...,216,52,0,18,0,2,500020,7105,517,515


## Party names

In [165]:
subfolder_par = "party_info"
path_par = os.path.join(folder, subfolder_par)

file_path_pol = os.path.join(path_par, "pol_strany.csv")
party_info = pd.read_csv(file_path_pol, sep=';')

party_info = party_info[['Kandidátní listina.číslo', 'Kandidátní listina.název']][1:]
party_info['Kandidátní listina.číslo'] = party_info['Kandidátní listina.číslo'].astype(int)
party_info['Kandidátní listina.název'] = party_info['Kandidátní listina.název'].str.replace('\u200b', '', regex=False).str.strip()
party_name_dict = dict(zip(party_info['Kandidátní listina.číslo'], party_info['Kandidátní listina.název']))

In [166]:
for col in df_okrsek.columns:
    if col in party_name_dict:
        df_okrsek = df_okrsek.rename(columns={col: party_name_dict[col]})

df_okrsek.head()

,ID_OKRSKY,Rebelové,MZH,JaSaN,VÝZVA,SMSka,SPD,ČSSD,PŘÍSAHA,Levice,...,ANO,STAN,Kruh,Stačilo!,Voluntia,OKRSEK,OBEC,OKRES,ODEVZ_OBAL,PL_HL_CELK
0,1,0,0,2,1,0,41,0,5,0,...,159,58,0,23,2,1,500011,7204,508,502
1,2,0,0,0,0,0,34,2,8,0,...,78,42,0,8,0,2,500011,7204,303,301
2,3,0,1,0,1,0,18,1,5,0,...,88,38,0,13,1,3,500011,7204,298,296
3,4,0,0,0,0,0,20,0,2,1,...,69,18,0,6,0,1,500020,7105,167,163
4,5,0,2,0,0,1,67,0,7,0,...,216,52,0,18,0,2,500020,7105,517,515


## Aggregate

In [167]:
df_okrsek = df_okrsek.drop(columns=['OKRSEK'])
df_obec = (df_okrsek.groupby('OBEC')
    .agg({col: ('sum' if col != 'OKRES' else 'first') for col in df_okrsek.columns}
))

## Percentage of votes received

In [168]:
non_party_col = ['OBEC', 'OKRES', 'ODEVZ_OBAL', 'PL_HL_CELK']
for col in df_obec.columns:
    if col not in non_party_col:
        df_obec[f"{col}_%"] = df_obec[col] / df_obec["PL_HL_CELK"] * 100
df_obec["NEPL_HLASY_%"] = 1 - df_obec["PL_HL_CELK"] / df_obec["ODEVZ_OBAL"]

In [169]:
df_obec.head()

,ID_OKRSKY,Rebelové,MZH,JaSaN,VÝZVA,SMSka,SPD,ČSSD,PŘÍSAHA,Levice,...,Volt_%,PB_%,AUTO_%,BPS_%,ANO_%,STAN_%,Kruh_%,Stačilo!_%,Voluntia_%,NEPL_HLASY_%
OBEC,,,,,,,,,,,,,,,,,,,,,
500011,6,0,1,2,2,0,93,3,18,0,...,0.000000,0.000000,6.824386,0.000000,29.572338,12.556870,0.000000,4.003640,0.272975,0.009017
500020,9,0,2,0,0,1,87,0,9,1,...,0.000000,0.000000,7.079646,0.000000,42.035398,10.324484,0.000000,3.539823,0.000000,0.008772
500046,6,0,0,1,0,2,63,2,13,1,...,0.000000,0.000000,7.509881,0.000000,46.245059,6.719368,0.000000,6.521739,0.000000,0.017476
500054,357,13,0,4,3,7,736,11,55,34,...,0.064562,0.121951,4.591105,0.143472,12.202296,13.507891,0.121951,2.295552,0.164993,0.020792
500062,28,0,0,0,1,3,100,0,13,1,...,0.080972,0.000000,11.012146,0.000000,32.550607,9.797571,0.000000,2.914980,0.080972,0.016720


# Demographic data

In [170]:
import numpy as np

In [171]:
subfolder_dem = "demographic"
path_dem = os.path.join(folder, subfolder_dem)

file_path_dem = os.path.join(path_dem, "dem_obec_2021.xlsx")
demog_imp = pd.read_excel(file_path_dem)

In [172]:
# Processing columns titles
demog = demog_imp.copy()
demog.columns = (
    demog.columns.to_series().replace(r"^věk:\s*(\d+)\s*-\s*(\d+)", r"\1-\2", regex=True)
)
demog.columns = (
    demog.columns.to_series().replace(r"^Unnamed.*", pd.NA, regex=True).ffill()
)
demog.iloc[1] = demog.iloc[1].replace({
    'muži': "M",
    "ženy": "Z",
    "abs.": "TOT"
})
demog.iloc[0] = demog.iloc[0].ffill()
demog.columns = (
    demog.columns.astype(str)
    + "_"
    + demog.iloc[1].astype(str)
)
demog = demog.drop(index=[0, 1])

# Renaming column titles
demog.columns = ["Obec_KOD", "Obec", "Pop"] + list(demog.columns[3:])
demog["Obec_KOD"] = demog["Obec_KOD"].astype(int)

# demog.head()
demog.columns

Index(['Obec_KOD', 'Obec', 'Pop', 'Obyvatelstvo celkem_%',
       'Obyvatelstvo celkem_M', 'Obyvatelstvo celkem_Z', '0-4_TOT', '0-4_%',
       '0-4_M', '0-4_Z',
       ...
       'vzdělání: vyšší odborné, konzervatoř_M',
       'vzdělání: vyšší odborné, konzervatoř_Z', 'vzdělání: vysokoškolské_TOT',
       'vzdělání: vysokoškolské_%', 'vzdělání: vysokoškolské_M',
       'vzdělání: vysokoškolské_Z', 'vzdělání: nezjištěno_TOT',
       'vzdělání: nezjištěno_%', 'vzdělání: nezjištěno_M',
       'vzdělání: nezjištěno_Z'],
      dtype='object', length=141)

# Combine

In [173]:
data = pd.merge(df_obec, demog, how = 'inner', left_index = True, right_on = 'Obec_KOD')

In [174]:
data.to_csv(os.path.join(folder, "prepared_data.csv"), index=False)

In [175]:
df_obec.head()

,ID_OKRSKY,Rebelové,MZH,JaSaN,VÝZVA,SMSka,SPD,ČSSD,PŘÍSAHA,Levice,...,Volt_%,PB_%,AUTO_%,BPS_%,ANO_%,STAN_%,Kruh_%,Stačilo!_%,Voluntia_%,NEPL_HLASY_%
OBEC,,,,,,,,,,,,,,,,,,,,,
500011,6,0,1,2,2,0,93,3,18,0,...,0.000000,0.000000,6.824386,0.000000,29.572338,12.556870,0.000000,4.003640,0.272975,0.009017
500020,9,0,2,0,0,1,87,0,9,1,...,0.000000,0.000000,7.079646,0.000000,42.035398,10.324484,0.000000,3.539823,0.000000,0.008772
500046,6,0,0,1,0,2,63,2,13,1,...,0.000000,0.000000,7.509881,0.000000,46.245059,6.719368,0.000000,6.521739,0.000000,0.017476
500054,357,13,0,4,3,7,736,11,55,34,...,0.064562,0.121951,4.591105,0.143472,12.202296,13.507891,0.121951,2.295552,0.164993,0.020792
500062,28,0,0,0,1,3,100,0,13,1,...,0.080972,0.000000,11.012146,0.000000,32.550607,9.797571,0.000000,2.914980,0.080972,0.016720


In [176]:
# Missing codes
set_demog = set(demog['Obec_KOD'].unique())
set_obec = set(df_obec['OBEC'].unique())

print("Obec codes (demographics):", len(set_demog))
print("Obec codes (votes):", len(set_obec))
print("Vote obec codes not in demograhics data:", len(set_obec - set_demog))

missing_votes = df_obec[~(df_obec.index.isin(demog['Obec_KOD']))]
missing_dem = demog[~(demog['Obec_KOD'].isin(df_obec.index))]
print("\nLargest obec population (from vote data not present in demographics data):\n", 
    missing_votes.sort_values(by = 'PL_HL_CELK', ascending= False)['PL_HL_CELK'][:5])
print("\n")
tot_pop = int(demog['Pop'].sum())
tot_vot = int(df_obec['PL_HL_CELK'].sum())
am_pop = int(data['Pop'].sum())
am_vot = int(data['PL_HL_CELK'].sum())

print('Total population:', tot_pop)
print('Total votes:', tot_vot)
print('After merge population:', am_pop, f"{am_pop/tot_pop*100:.1f}%")
print('After merge votes:', am_vot, f"{am_vot/tot_vot*100:.1f}%")

Obec codes (demographics): 6254
Obec codes (votes): 6388
Vote obec codes not in demograhics data: 1488

Largest obec population (from vote data not present in demographics data):
 OBEC
500119    63855
500178    55271
500224    54365
500208    51345
556904    47257
Name: PL_HL_CELK, dtype: int64


Total population: 10524167
Total votes: 5593772
After merge population: 8028950 76.3%
After merge votes: 4376613 78.2%
